- 단어 사전 생성: 전체 텍스트 데이터에서 고유한 단어들을 수집하여 각 단어에 고유한 인덱스를 매핑합니다.

- 토큰화 및 인덱스 변환: 각 텍스트를 토큰화한 후, 생성한 단어 사전을 사용하여 토큰을 해당 인덱스로 변환합니다.

- 커스텀 데이터셋 생성: 변환된 인덱스를 사용하여 PyTorch의 Dataset 클래스를 상속받은 커스텀 데이터셋 클래스를 만듭니다.

- 데이터로더 생성: 마지막으로 DataLoader를 사용하여 배치 처리를 수행할 수 있도록 준비합니다.

### 0. 모듈 로딩

In [44]:
from Korpora import Korpora
from konlpy.tag import Okt, Kkma
import spacy
from collections import Counter

import torch
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import re

### 1. 데이터 불러오기

In [45]:
# NSMC 데이터 불러오기
nsmc = Korpora.load("nsmc")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at /Users/anhyojun/Korpora/nsmc/ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at /Users/anhy

### 2. 토크나이저 만들기

In [46]:
# Okt 형태소 분석기
okt = Okt()

# 테스트 문장에 적용
sample_text = nsmc.test[150].text
tokens = okt.morphs(sample_text)
print(tokens)

['다시', '봐도', '수작', '이다', '.', '미국', '국수', '주의', '라', '뭐라는', '시점', '도', '있겠으나', '전쟁', '물', '중', '에', '손가락', '안', '에', '꼽', '을', '정도', '로', '잘', '만들어', '졌다', '.', '스토리', '라인', '은', '뻔하나', '그', '뻔한', '걸', '너무', '잘', '살렸다', '.', '언제', '봐도', '다시', '봐도', '재미', '있다', '.', '10년', '이', '지난', '영화', '가', '..']


### 3. 불용어 처리

In [47]:
with open('stopword.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# 불용어 목록
print(stopwords)

['이', '그', '저', '그리고', '하지만', '그래서', '또한', '즉', '만약', '이러한', '그런', '이런', '때문에', '위해', '대해', '속', '곳', '더욱', '을', '를', '의', '에', '이다', '라', '도', '중', '로', '은', '걸', '있다', '가', '들', '만', '어', '요', '됨', '움', '는', '냐', '와', '적', '인', '때', '름', '이건', '이네', '까지', '엉', '고', '작', '말', '라는', '게', '내', '것', '따', '수', '랄', '까요', '하네요', '임', '인가', '하지', '돼', '감', '바', '당', '하', '드', '네', '이에요', '있다나', '들에게', '하면서', '나왔는데', '어디', '갔나', '렷습', '니', '다', '롭', '일', '뿐', '나', '대', '라서', '한테', '두', '막', '분', '알', '으로부터', '받은', '였다', '본', '라도', '이며', '문', '가라', 'ㅈ', '평', '해', '몇', '얼', '등', '력', '아', '한', '에서', '된', '마치', '개']


In [48]:
# 불용어 제거 함수
def remove_stopwords(tokens, stopwords):
    return [token for token in tokens if token not in stopwords]

# 구두점 제거 함수
def remove_punctuation(tokens):
    # match는 문장의 처음부터 매칭돼야 함
    return [token for token in tokens if re.match(r'[\w가-힇]+', token)]

In [49]:
clean_tokens = remove_stopwords(tokens, stopwords)
clean_tokens = remove_punctuation(clean_tokens)
print("불용어 제거 결과:", clean_tokens)

불용어 제거 결과: ['다시', '봐도', '수작', '미국', '국수', '주의', '뭐라는', '시점', '있겠으나', '전쟁', '물', '손가락', '안', '꼽', '정도', '잘', '만들어', '졌다', '스토리', '라인', '뻔하나', '뻔한', '너무', '잘', '살렸다', '언제', '봐도', '다시', '봐도', '재미', '10년', '지난', '영화']


### 4. 문장 길이 맞추기 (패딩)

In [50]:
max_length = 10
padding_token = '<PAD>'

def pad_sequence(tokens, max_length, padding_token, cut_front=False):
    # 뒷부분을 자를 경우
    if cut_front == False:
        # 토큰 길이가 max_length보다 짧은 경우
        if len(tokens) < max_length:
            return tokens + [padding_token]*(max_length - len(tokens))
        # 토큰 길이가 max_length보다 긴 경우
        else:
            return tokens[:max_length]
    # 앞부분을 자를 경우
    elif cut_front == True:
        # 토큰 길이가 max_length보다 짧은 경우
        if len(tokens) < max_length:
            return [padding_token]*(max_length - len(tokens)) + tokens
        # 토큰 길이가 max_length보다 긴 경우
        else:
            return tokens[(max_length - len(tokens)):]

padded_token = pad_sequence(clean_tokens, max_length, padding_token)
print("패딩 적용 결과:", padded_token)

패딩 적용 결과: ['다시', '봐도', '수작', '미국', '국수', '주의', '뭐라는', '시점', '있겠으나', '전쟁']


### 5. 커스텀 데이터셋 생성

In [51]:
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        return text, label
    
texts = nsmc.test.texts
labels = nsmc.test.labels

dataset = TextDataset(texts, labels)

DL = DataLoader(dataset, batch_size = 2)

In [52]:
for i in DL:
    print(i)
    break

[('굳 ㅋ', 'GDNTOPCLASSINTHECLUB'), tensor([1, 0])]


### 6. 사전(Vocab) 생성

In [53]:
def build_voca(DL, stopwords, tokenizer = Okt().morphs, padding_token = '<PAD>', unknown_token = '<UNK>'):
    counter = Counter()
    for texts, labels in DL:
        for text in texts:
            # 텍스트 토큰화
            tokens = tokenizer(text)

            # 불용어 및 구두점 제거
            clean_tokens = remove_stopwords(tokens, stopwords)
            clean_tokens = remove_punctuation(clean_tokens)

            counter.update(clean_tokens)

    vocab = {padding_token : 0, unknown_token : 1}
    vocab.update({word : idx+2 for idx, (word, freq) in enumerate(counter.items())})

    return vocab

In [54]:
def spacy_tokenizer(text):
    nlp = spacy.load('ko_core_news_sm')
    # spacy 문서로 변환
    doc = nlp(text)
    # 각 토큰의 텍스트를 리스트로 변환
    return [token.text for token in doc]

In [55]:
# 테스트
string = '오늘 날씨가 참 맑구나. 내일은 흐릴 수도 있겠는데?'

okt = Okt().morphs
print(okt(string))

kkma = Kkma().morphs
print(kkma(string))

print(spacy_tokenizer(string))

['오늘', '날씨', '가', '참', '맑구나', '.', '내일', '은', '흐릴', '수도', '있겠는데', '?']
['오늘', '날씨', '가', '참', '맑', '구나', '.', '내일', '은', '흐리', 'ㄹ', '수', '도', '있', '겠', '는데', '?']
['오늘', '날씨가', '참', '맑구나', '.', '내일은', '흐릴', '수도', '있겠는데', '?']


In [56]:
okt_voca = build_voca(DL, stopwords, tokenizer = Okt().morphs)
# kkma_voca = build_voca(DL, stopwords, tokenizer = Kkma().morphs) # Kkma 같은 Java 기반 라이브러리는 메모리 오류 많이 뜸
# spacy_voca = build_voca(DL, stopwords, tokenizer = spacy_tokenizer)

In [57]:
okt_voca

{'<PAD>': 0,
 '<UNK>': 1,
 '굳': 2,
 'ㅋ': 3,
 'GDNTOPCLASSINTHECLUB': 4,
 '뭐': 5,
 '야': 6,
 '평점': 7,
 '나쁘진': 8,
 '않지만': 9,
 '10': 10,
 '점': 11,
 '짜': 12,
 '리': 13,
 '더': 14,
 '아니잖아': 15,
 '지루하지는': 16,
 '않은데': 17,
 '완전': 18,
 '막장': 19,
 '돈': 20,
 '주고': 21,
 '보기': 22,
 '에는': 23,
 '3': 24,
 'D': 25,
 '아니었어도': 26,
 '별': 27,
 '다섯': 28,
 '줬을텐데': 29,
 '왜': 30,
 '나와서': 31,
 '제': 32,
 '심기': 33,
 '불편하게': 34,
 '하죠': 35,
 '음악': 36,
 '주가': 37,
 '최고': 38,
 '영화': 39,
 '진정한': 40,
 '쓰레기': 41,
 '미국': 42,
 '애니': 43,
 '튀어나온듯': 44,
 '창의력': 45,
 '없는': 46,
 '로봇': 47,
 '디자인': 48,
 '부터가': 49,
 '고개': 50,
 '젖게': 51,
 '한다': 52,
 '갈수록': 53,
 '개판': 54,
 '되가는': 55,
 '중국영화': 56,
 '유치하고': 57,
 '내용': 58,
 '없음': 59,
 '폼': 60,
 '잡다': 61,
 '끝남': 62,
 '안되는': 63,
 '무기': 64,
 '유치한': 65,
 'cg': 66,
 '남무': 67,
 '그립다': 68,
 '동사서독': 69,
 '같은': 70,
 '류': 71,
 '류작': 72,
 '이별': 73,
 '아픔': 74,
 '뒤': 75,
 '찾아오는': 76,
 '새로운': 77,
 '인연': 78,
 '기쁨': 79,
 'But': 80,
 '모든': 81,
 '사람': 82,
 '그렇지는': 83,
 '않네': 84,
 '괜찮네요': 85,
 '오랜': 86,
 '포

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from konlpy.tag import Okt
from torch.nn.utils.rnn import pad_sequence

class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
        self.okt = Okt()
        self.word_to_index = self.build_vocabulary(texts)

    def build_vocabulary(self, texts):
        # 모든 텍스트에서 고유한 단어 추출
        unique_words = set()
        for text in texts:
            tokens = self.okt.morphs(text)
            unique_words.update(tokens)
        # 인덱스를 할당
        return {word: idx for idx, word in enumerate(unique_words, start=1)}  # 1부터 시작

    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # 토큰화
        tokens = self.okt.morphs(text)
        # 인덱스 변환
        indexed_tokens = [self.word_to_index[word] for word in tokens]
        
        # 텐서로 변환
        token_tensor = torch.tensor(indexed_tokens, dtype=torch.long)
        return token_tensor, label

# 예시 데이터
texts = ['안녕하세요. 반갑습니다.', '오늘 날씨가 참 좋네요.']
labels = [1, 0]

# 데이터셋과 데이터로더 생성
dataset = TextDataset(texts, labels)
DL = DataLoader(dataset, batch_size=2)

# 데이터 출력 예시
for batch in DL:
    print(batch)
